In [71]:
from urllib.request import urlretrieve
from urllib.parse import urlencode
import image_slicer
import traceback
from os import listdir, remove, mkdir
from os.path import join, exists, expanduser
from shutil import rmtree
from PIL import Image

In [72]:
def remove_files(dir_path):
    for file_name in listdir(dir_path):
        if file_name.endswith('.jpg'):
            remove(join(dir_path, file_name))
    return False

In [73]:
def make_dir_helper(dir_path):
    if not exists(dir_path):
        mkdir(dir_path)
        return True
    return False

In [74]:
def crop(map_file_path, sliced_dir, height, width):
    with Image.open(map_file_path) as im:
        imgwidth, imgheight = im.size
        idx = 0
        for i in range(0,imgheight - height , height): # Don't include borders
            for j in range(0,imgwidth - width , width):
                box = (j, i, j+width, i+height)
                try:
                    a = im.crop(box)
                    a.save(join(sliced_dir, 'image_{}.jpg'.format(idx)), "JPEG", quality=100, optimize=True)
                except:
                    traceback.print_exc()
                    pass
                idx +=1

In [75]:
project_dir = join(expanduser('~'),'desktop','Esri Project');

In [76]:
URL = 'http://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/export?'
x_min = -117.261067
y_min = 33.052972
x_max = -117.256993
y_max = 33.054659
spatial_ref = 4326
width = 1000
height = 1000

parameters = {
    'bbox':'{},{},{},{}'.format(x_min, y_min, x_max, y_max),
    'bboxSR':spatial_ref,
    'size':'{},{}'.format(width, height),
    'format':'jpg',
    'f':'image'
}

URL += urlencode(parameters)

MAP_NAME = 'map.jpg'
MAP_DIR = join(project_dir, 'maps')
SLICED_DIR = join(MAP_DIR, 'sliced')
make_dir_helper(MAP_DIR)
make_dir_helper(SLICED_DIR)

MAP_FILE_PATH = join(MAP_DIR, MAP_NAME)

urlretrieve(URL, MAP_FILE_PATH)

tile_size = 200
remove_files(SLICED_DIR)
crop(MAP_FILE_PATH, SLICED_DIR, tile_size, tile_size)